# 1. Import relevant libraries

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


import matplotlib.cm as cm
import matplotlib.colors as colors





import folium

print("Libraries imported.")

Libraries imported.


# 2. Data scraping from Wikipedia

In [2]:
data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

The above is a get request which scrapes text from the webpage

In [3]:
soup=BeautifulSoup(data,'html.parser')

In [4]:
postal_code=[]
borough=[]
neighborhood=[]

### BeautifulSoup functions

In HTML, tables are marked as 'tr'

*  soup.find('table').find_all('tr') finds all tables in the webpage 

In HTML, all table rows are marked as 'td'. Hence, to obtain the data, we need to go through each td instance as shown below

* for rows in soup.find('table').find_all('tr'):


    cells=rows.find_all('td')


In [5]:
for row in soup.find('table').find_all('tr'):
    cells=row.find_all('td')
    if (len(cells)>0):
        postal_code.append(cells[0].text)
        borough.append(cells[1].text)
        neighborhood.append(cells[2].text)

### Lets take a look at the lists we have scrapped in the webpage

In [6]:
postal_code_list=[]
for string in postal_code:
    postal_code_list.append(string.replace('\n',''))
    
postal_code_list[0:10]

['M1A', 'M2A', 'M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M8A', 'M9A', 'M1B']

In [7]:
borough[0:10]
borough_list=[]

for b in borough:
    borough_list.append(b[0:-1])

borough_list[0:10]

['Not assigned',
 'Not assigned',
 'North York',
 'North York',
 'Downtown Toronto',
 'North York',
 'Downtown Toronto',
 'Not assigned',
 'Etobicoke',
 'Scarborough']

In [8]:
neighborhood[0:10]
neighborhood_list=[]

for n in neighborhood:
    neighborhood_list.append(n[0:-1])
neighborhood_list[0:10]

['Not assigned',
 'Not assigned',
 'Parkwoods',
 'Victoria Village',
 'Regent Park, Harbourfront',
 'Lawrence Manor, Lawrence Heights',
 "Queen's Park, Ontario Provincial Government",
 'Not assigned',
 'Islington Avenue, Humber Valley Village',
 'Malvern, Rouge']

### Now, let us transform the above lists into a pandas dataframe for better readability

# 3. Dataframe creation

In [9]:
df=pd.DataFrame({'Postal code':postal_code_list,'Borough':borough_list,'Neighborhood':neighborhood_list})
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


 However, we need to pre process the data since we have a lot of "Not assigned" in the Borough and Neighborhood columns

In [10]:
(df['Neighborhood']=='Not assigned').sum()

77

In [11]:
(df['Borough']=='Not assigned').sum()

77

We shall drop all the entries where Borough name is Not assigned. If Borough name is assigned but neighborhood in not, then Borough name is Neighborhood name

In [12]:
df_processed=df.copy()
df_processed=df[df['Borough']!='Not assigned']
df_processed

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [13]:
(df_processed['Neighborhood']=='Not assigned').any()

False

Hence, it can be seen that after clearing the unassigned boroughs, there are no unassigned neighborhoods

# 4. Grouping the rows with same postal code 

In [14]:
df_grouped=df_processed.groupby(['Postal code','Borough'],as_index=False).agg(lambda x: ', '.join(x))
df_grouped.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# 5. Unassigned neighborhoods as Boroughs

Although thankfully we did face the issue of having unassigned neighborhoods after removing all the unassigned Boroughs, we can test the code for taking care of the unassigned neighborhoods as follows.

If in case the neighborhood is unassigned, it must be the same as the borough name. An example has been shown below.

In [15]:
df['Borough'][0]='sample1'
df['Borough'][1]='sample2'

for index,rows in df.iterrows():
    if rows['Neighborhood']=='Not assigned':
        rows['Neighborhood']=rows['Borough']
           

In [16]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,sample1,sample1
1,M2A,sample2,sample2
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# 6. Comparing the dataframe

In [17]:
postal_test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

df_test=pd.DataFrame(columns=df_grouped.columns)

for pcodes in postal_test_list:
    df_test=df_test.append(df_grouped[df_grouped['Postal code']==pcodes])

In [18]:
df_test.reset_index(inplace=True)

In [19]:
df_test.drop('index',axis=1,inplace=True)

In [20]:
df_test

,Postal code,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


# 7. Merging postal codes with Latitudes and Longitudes

In [21]:
df_lat_lon=pd.read_csv('Geospatial_Coordinates.csv')
df_lat_lon.rename(columns={'Postal Code':'Postal code'},inplace=True)
df_lat_lon.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


As we can see, the latitudes and longitudes are postal coded uniquely. Hence, we can merge our data frame by postal codes using the pandas merge function.

In [22]:
df_locations=df_grouped.merge(df_lat_lon, on='Postal code')
df_locations.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Let us check specifically for the postal codes which were asked in the assignment.

In [23]:
df_test1=pd.DataFrame(columns=df_locations.columns)

for pcodes in postal_test_list:
    df_test1=df_test1.append(df_locations[df_locations['Postal code']==pcodes],ignore_index=True)

In [24]:
df_test1

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420


# 8. Creation of Toronto map

From the above dataframe, we can estimate the latitudes and logitudes of Toronto. We could use geolocator as suggested by Coursera aswell.

In [25]:
#Toronto coordinates

latitude=43.65
longitude=-79.4

map_toronto=folium.Map(location=[latitude,longitude],zoom_start=11)
map_toronto

In [26]:
for lat,lon,borough,neighborhood in zip(df_locations['Latitude'],df_locations['Longitude'],df_locations['Borough'],df_locations['Neighborhood']):
                                        
                                        label='{} {}'.format(neighborhood,borough)
                                        label=folium.Popup(label)
                                        
                                        folium.CircleMarker(
                                            [lat,lon], radius=5,popup=label,color='orange',fill=True,fill_color='black',fill_opacity=0.6).add_to(map_toronto)
                                   

In [27]:
map_toronto

# 9. Downsizing the dataframe


In [28]:
boroughs=list(df_locations['Borough'].unique())
toronto_boroughs=[]

for names in boroughs:
    if 'toronto' in names.lower():
        toronto_boroughs.append(names)
        
toronto_boroughs

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [29]:
df_toronto=pd.DataFrame(columns=df_locations.columns)

for names in toronto_boroughs:
    df_toronto=df_toronto.append(df_locations[df_locations['Borough']==names],ignore_index=True)
    
df_toronto.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


# 10. Using foursquare for location exploration

In [30]:
CLIENT_ID='ZALY0GYPRA1XXX0BZZQNLFG1C3FBIFA4E23TL4XPUSQ42G1P'
CLIENT_SECRET='XA524DRFQCWEIRUGCIHPVUOADC32EPK3OPCHFRSUERFXCPWH'
VERSION = '20180605'

Let us explore the top 10 places within a 500 meter radius

In [31]:
radius = 500
LIMIT = 10

venues = []

for lat, lon, pcode, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Postal code'], df_toronto['Borough'], df_toronto['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lon,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            pcode, 
            borough,
            neighborhood,
            lat, 
            lon, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [32]:
venues_df=pd.DataFrame(venues)
venues_df.head()

,0,1,2,3,4,5,6,7,8
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [33]:
venues_df.rename(columns={0:'Postal code',1:'Borough',2:'Neighborhood',3:'Borough latitude',4:'Borough longitude',5:'Venue name',6:'Venue latitude',7:'Venue longitude',8:'Category'},inplace=True)
venues_df.head()

,Postal code,Borough,Neighborhood,Borough latitude,Borough longitude,Venue name,Venue latitude,Venue longitude,Category
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


# 11. Searching for a specific venue category

In [37]:
print('Number of unique categories of venues : {}'.format(len(venues_df)))

Number of unique categories of venues : 348


As we can see, there are about 348 venue categories which are a lot. If we are interested in a Steakhouse, we can search for a particular query as shown below:

In [50]:
search_query='Steakhouse'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [51]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ede6b7cfb34b5001b486051'},
 'response': {'venues': [{'id': '54256356498edef2967366eb',
    'name': 'The Keg Steakhouse + Bar - King West',
    'location': {'address': '560 King St W',
     'lat': 43.64477859109389,
     'lng': -79.3992247945535,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64477859109389,
       'lng': -79.3992247945535}],
     'distance': 584,
     'postalCode': 'M5V 0L5',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['560 King St W', 'Toronto ON M5V 0L5', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'shortName': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1591634989',
    'hasPerk': False},
   {'id': '55fedf4c498e47bc0c154f91',
    'name'

In [52]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,54256356498edef2967366eb,The Keg Steakhouse + Bar - King West,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1591634989,False,560 King St W,43.644779,-79.399225,"[{'label': 'display', 'lat': 43.64477859109389...",584,M5V 0L5,CA,Toronto,ON,Canada,"[560 King St W, Toronto ON M5V 0L5, Canada]",NaN
1,55fedf4c498e47bc0c154f91,The Keg Steakhouse + Bar,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1591634989,False,560 King Street West,43.646760,-79.395368,"[{'label': 'display', 'lat': 43.64676043755301...",518,M5V 0L5,CA,Toronto,ON,Canada,"[560 King Street West, Toronto ON M5V 0L5, Can...",NaN
2,4b22f410f964a520005124e3,Jacobs & Co.,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1591634989,False,12 Brant St,43.645339,-79.398020,"[{'label': 'display', 'lat': 43.64533925830643...",542,M5V 2M1,CA,Toronto,ON,Canada,"[12 Brant St (King), Toronto ON M5V 2M1, Canada]",King


# 12. Data analysis

Let us check the various categories of venues returned

In [34]:
venues_df['Category'].unique()

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Park', 'Liquor Store',
       'Pet Store', 'Bookstore', 'Fish Market', 'Sandwich Place',
       'Bakery', 'Coffee Shop', 'Gay Bar', 'Farmers Market', 'Comic Shop',
       'Burrito Place', 'Pizza Place', 'Restaurant', 'Skate Park',
       'Garden Center', 'Auto Workshop', 'Swim School', 'Bus Line',
       'Breakfast Spot', 'Food & Drink Shop', 'Department Store', 'Hotel',
       'Gym / Fitness Center', 'Diner', 'Salon / Barbershop',
       'Mexican Restaurant', 'Spa', 'Clothing Store',
       'Chinese Restaurant', 'Dessert Shop', 'Café', 'Indian Restaurant',
       'Seafood Restaurant', 'Summer Camp', 'Tennis Court', 'Lawyer',
       'Supermarket', 'American Restaurant', 'Sports Bar',
       'Fried C

Let us check the number of venues returned for each postal code

In [36]:
venues_df.groupby(['Postal code','Borough','Neighborhood']).count()

,,,Borough latitude,Borough longitude,Venue name,Venue latitude,Venue longitude,Category
Postal code,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,4,4,4,4,4,4
M4K,East Toronto,"The Danforth West, Riverdale",10,10,10,10,10,10
M4L,East Toronto,"India Bazaar, The Beaches West",10,10,10,10,10,10
M4M,East Toronto,Studio District,10,10,10,10,10,10
M4N,Central Toronto,Lawrence Park,3,3,3,3,3,3
M4P,Central Toronto,Davisville North,8,8,8,8,8,8
M4R,Central Toronto,"North Toronto West, Lawrence Park",10,10,10,10,10,10
M4S,Central Toronto,Davisville,10,10,10,10,10,10
M4T,Central Toronto,"Moore Park, Summerhill East",4,4,4,4,4,4


As we can see from the data above, most postal codes show the top 10 places. Some of the postal codes lack enough venues.

Let us one hot encode the above dataframefor the venue categories field.

In [53]:
toronto_onehot=pd.get_dummies(venues_df['Category'])

In [57]:
venues_df=venues_df.iloc[:,:-1]

In [60]:
venues_df=venues_df.merge(toronto_onehot,on=venues_df.index)
venues_df

,key_0,Postal code,Borough,Neighborhood_x,Borough latitude,Borough longitude,Venue name,Venue latitude,Venue longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Lawyer,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood_y,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,6,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,0,0,

In [61]:
venues_df.drop('key_0',axis=1,inplace=True)
venues_df.head()

,Postal code,Borough,Neighborhood_x,Borough latitude,Borough longitude,Venue name,Venue latitude,Venue longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Lawyer,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood_y,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [67]:
venues_df.rename(columns={'Neighborhood_x':'Neighborhood'},inplace=True)
venues_df.head()

,Postal code,Borough,Neighborhood,Borough latitude,Borough longitude,Venue name,Venue latitude,Venue longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Lawyer,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood_y,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


We see that one postal code has various entries making the dataframe extremely large. Let us downsize the dataframe using groupby method.

In [122]:
venues_grouped=venues_df.groupby(['Postal code','Borough','Neighborhood']).mean().reset_index()

In [123]:
venues_grouped.head()

,Postal code,Borough,Neighborhood,Borough latitude,Borough longitude,Venue latitude,Venue longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Lawyer,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood_y,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,43.678861,-79.295440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,43.677804,-79.350938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.2,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.1
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668998,-79.315572,43.667437,-79.315405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.00,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.100000,0.0,0.1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,M4M,East Toronto,Studio District,43.659525,-79.340923,43.661016,-79.340683,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Let us check the concised dataframe size.

In [124]:
venues_grouped.shape

(39, 130)

Let us remove the latitudes and longitudes to simplify the dataframe further.

In [85]:
venues_grouped.drop(['Borough latitude','Borough longitude','Venue latitude','Venue longitude'],axis=1,inplace=True)

In [86]:
venues_grouped.head()

,Postal code,Borough,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Lawyer,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood_y,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0
1,M4K,East Toronto,"The Danforth West, Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.2,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.1
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.00,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.100000,0.0,0.1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0

Let us find out the top 5 venue categories of every postal code, borough and neighborhood as shown below.

In [106]:
num_top_venues=5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Postal code', 'Borough', 'Neighborhood']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most common venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most common venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal code'] = venues_grouped['Postal code']
neighborhoods_venues_sorted['Borough'] = venues_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']



for ind in np.arange(venues_grouped.shape[0]):
    row_categories=venues_grouped.iloc[ind,:].iloc[3:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind,3:]=row_categories_sorted.index.values[0:num_top_venues]

neighborhoods_venues_sorted.sort_values(freqColumns,inplace=True) 
    
neighborhoods_venues_sorted.reset_index(inplace=True,drop=True)
neighborhoods_venues_sorted

,Postal code,Borough,Neighborhood,1st Most common venue,2nd Most common venue,3rd Most common venue,4th Most common venue,5th Most common venue
0,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport,Airport Terminal,Harbor / Marina,Coffee Shop
1,M6H,West Toronto,"Dufferin, Dovercourt Village",Bakery,Middle Eastern Restaurant,Grocery Store,Music Venue,Bar
2,M4M,East Toronto,Studio District,Bookstore,Bakery,Pet Store,Gay Bar,Coffee Shop
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",Breakfast Spot,Restaurant,Historic Site,Park,Bakery
4,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",Café,Coffee Shop,Museum,Bakery,Restaurant
5,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",Café,Coffee Shop,Tea Room,Beer Bar,Bakery
6,M4X,Downtown Toronto,"St. James Town, Cabbagetown",Café,General Entertainment,Bakery,Diner,Restaurant
7,M6G,Downtown Toronto,Christie,Café,Grocery Store,Restaurant,Coffee Shop,Diner
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",Café,Park,Donut Shop,Vegetarian / Vegan Restaurant,Burger Joint
9,M5B,Downtown Toronto,"Garden District, Ryerson",Café,Tea Room,Comic Shop,Music Venue,Clothing Store


# 13. Clustering the venues into labels

Let us use 5 clusters to label the data.

In [111]:
from sklearn.cluster import KMeans
K=5

cluster_df=venues_grouped.drop(['Postal code', 'Borough', 'Neighborhood'],axis=1)
cluster_df.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Beer Store,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Café,Candy Store,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Department Store,Dessert Shop,Diner,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Lawyer,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood_y,New American Restaurant,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Skating Rink,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.2,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.00,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.100000,0.0,0.1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In order to apply any machine learning, we need to take care of all categorical data. The data frame must have only numerical data and must be properly one hot encoded.

In [114]:
kmeans=KMeans(n_clusters=K).fit(cluster_df)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [119]:
kmeans.labels_

array([0, 4, 4, 4, 0, 3, 4, 1, 1, 1, 0, 1, 3, 3, 1, 4, 3, 1, 3, 3, 1, 1,
       2, 0, 1, 4, 1, 1, 3, 1, 1, 1, 4, 1, 4, 1, 1, 4, 4], dtype=int32)

Let us fit the labels into a dataframe for better readability

In [121]:
labelled_df=venues_grouped.iloc[:,:3]
labelled_df['Cluster label']=kmeans.labels_
labelled_df

,Postal code,Borough,Neighborhood,Cluster label
0,M4E,East Toronto,The Beaches,0
1,M4K,East Toronto,"The Danforth West, Riverdale",4
2,M4L,East Toronto,"India Bazaar, The Beaches West",4
3,M4M,East Toronto,Studio District,4
4,M4N,Central Toronto,Lawrence Park,0
5,M4P,Central Toronto,Davisville North,3
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",4
7,M4S,Central Toronto,Davisville,1
8,M4T,Central Toronto,"Moore Park, Summerhill East",1
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",1


We have clustered the various venues into 5 labels. Let us visualise these venues on a folium map of Toronto. We also add the venue latitudes and longitudes to the above dataframe for map plotting

In [125]:
labelled_df['Latitude']=venues_grouped['Venue latitude']
labelled_df['Longitude']=venues_grouped['Venue longitude']
labelled_df.head()

,Postal code,Borough,Neighborhood,Cluster label,Latitude,Longitude
0,M4E,East Toronto,The Beaches,0,43.678861,-79.295440
1,M4K,East Toronto,"The Danforth West, Riverdale",4,43.677804,-79.350938
2,M4L,East Toronto,"India Bazaar, The Beaches West",4,43.667437,-79.315405
3,M4M,East Toronto,Studio District,4,43.661016,-79.340683
4,M4N,Central Toronto,Lawrence Park,0,43.727840,-79.386683


In [137]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(K)
ys = [i+x+(i*x)**2 for i in range(K)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, pcode, bor, nei, cluster in zip(labelled_df['Latitude'], labelled_df['Longitude'], labelled_df['Postal code'], labelled_df['Borough'], labelled_df['Neighborhood'], labelled_df['Cluster label']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, pcode, nei, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

We see the various clusters marked according to their locations.

# 14. Checking the various dataframes cluster by cluster

## Cluster 1

In [130]:
first_cluster_df=labelled_df.loc[labelled_df['Cluster label'].isin([0])]
first_cluster_df

,Postal code,Borough,Neighborhood,Cluster label,Latitude,Longitude
0,M4E,East Toronto,The Beaches,0,43.678861,-79.295440
4,M4N,Central Toronto,Lawrence Park,0,43.727840,-79.386683
10,M4W,Downtown Toronto,Rosedale,0,43.679754,-79.377335
23,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",0,43.699872,-79.408077


## Cluster 2

In [131]:
second_cluster_df=labelled_df.loc[labelled_df['Cluster label'].isin([1])]
second_cluster_df

,Postal code,Borough,Neighborhood,Cluster label,Latitude,Longitude
7,M4S,Central Toronto,Davisville,1,43.704647,-79.388631
8,M4T,Central Toronto,"Moore Park, Summerhill East",1,43.689155,-79.384125
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",1,43.687648,-79.396557
11,M4X,Downtown Toronto,"St. James Town, Cabbagetown",1,43.666833,-79.368773
14,M5B,Downtown Toronto,"Garden District, Ryerson",1,43.656542,-79.380004
17,M5G,Downtown Toronto,Central Bay Street,1,43.658421,-79.385060
20,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",1,43.647780,-79.380680
21,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",1,43.647980,-79.380275
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",1,43.674543,-79.407563
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",1,43.654236,-79.400258


## Cluster 3

In [133]:
third_cluster_df=labelled_df.loc[labelled_df['Cluster label'].isin([2])]
third_cluster_df

,Postal code,Borough,Neighborhood,Cluster label,Latitude,Longitude
22,M5N,Central Toronto,Roselawn,2,43.712189,-79.411978


## Cluster 4

In [135]:
fourth_cluster_df=labelled_df.loc[labelled_df['Cluster label']==3]
fourth_cluster_df

,Postal code,Borough,Neighborhood,Cluster label,Latitude,Longitude
5,M4P,Central Toronto,Davisville North,3,43.711905,-79.391119
12,M4Y,Downtown Toronto,Church and Wellesley,3,43.666288,-79.383265
13,M5A,Downtown Toronto,"Regent Park, Harbourfront",3,43.654015,-79.359960
16,M5E,Downtown Toronto,Berczy Park,3,43.646835,-79.373960
18,M5H,Downtown Toronto,"Richmond, Adelaide, King",3,43.650173,-79.384718
19,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",3,43.641124,-79.381885
28,M5W,Downtown Toronto,Stn A PO Boxes,3,43.647805,-79.375250


## Cluster 5

In [136]:
fifth_cluster_df=labelled_df.loc[labelled_df['Cluster label']==4]
fifth_cluster_df

,Postal code,Borough,Neighborhood,Cluster label,Latitude,Longitude
1,M4K,East Toronto,"The Danforth West, Riverdale",4,43.677804,-79.350938
2,M4L,East Toronto,"India Bazaar, The Beaches West",4,43.667437,-79.315405
3,M4M,East Toronto,Studio District,4,43.661016,-79.340683
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",4,43.715059,-79.400213
15,M5C,Downtown Toronto,St. James Town,4,43.651302,-79.374467
25,M5S,Downtown Toronto,"University of Toronto, Harbord",4,43.662756,-79.403533
32,M6J,West Toronto,"Little Portugal, Trinity",4,43.647175,-79.420034
34,M6P,West Toronto,"High Park, The Junction South",4,43.664684,-79.465895
37,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",4,43.662893,-79.385930
38,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",4,43.664277,-79.322027


## Cluster sizes

In [139]:
cluster_size_df=pd.DataFrame(columns=['Cluster label','size'])
clusters=np.arange(1,6)
cluster_size_df['Cluster label']=clusters


In [143]:
sizes=np.array([first_cluster_df.shape[0],second_cluster_df.shape[0],third_cluster_df.shape[0],fourth_cluster_df.shape[0],fifth_cluster_df.shape[0]])

In [145]:
cluster_size_df['size']=sizes

In [146]:
cluster_size_df

,Cluster label,size
0,1,4
1,2,17
2,3,1
3,4,7
4,5,10


As we can see, cluster 2 is the biggest while cluster 3 has only 1 entry.